# Дообучение GPT-2 для генерации текстов песен метал групп.

В этом ноутбуке представлен fine-tuning модели GPT-2 для генерации тесктов песен метал групп на английском языке. Для работы с моделью GPT-2 используется библиотека Hugging Face Transformers.


## Подготовка

In [1]:
from tqdm import tqdm

import pandas as pd
import numpy as np
import random

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [2]:
# если ноутбук запускается в колабе, лучше подключить google drive для сохранения весов модели
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Задаем параметры обучения

In [ ]:
args = {
    "batch_size": 2,        # размер батча (размер 2 был выбран, так как при большем размере память GPU переполняется)
    "train_size": 0.8,      # доля тренеровочной выборки от всего датасета
    "epochs": 5,            # количество эпох обучения
    "learning_rate": 5e-4,  # темп обучения модели
    "warmup_steps": 100,    # количество шагов, после которого шедуллер будет уменьшать learning rate
}

Загружаем датасет

In [ ]:
data = pd.read_csv("dataset.csv")["lyrics"]
data

,lyrics
0,Leave me alone\nLike a dog with a bone\nLike a...
1,"""The following is a true story, only the names..."
2,"""Get ready\nYou think your kinda tough\nYou're..."
3,Do you wanna boogie?\nDo you wanna blow?\nShe ...
4,"Got a taste of a rocking band,\nStanding there..."
...,...
10541,I make a call\nSo far to fall\nRestless cravin...
10542,Periodic tableware\nPsychotropic science fare\...
10543,The 28th day\nShe'll be bleeding again\nAnd in...
10544,She thinks I'm iron man that I don't feel pain...


In [ ]:
data = data.sample(len(data))

In [4]:
# max_len = max([len(text) for text in data])
max_len = 6290
print("максимальная длина текста в датасете:", max_len)

максимальная длина текста в датасете: 6290


Загружаем токенизатор для GPT-2

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>', max_length=6290)

d:\anaconda\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Описываем класс датасета

In [ ]:
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    # токенизируем текста
    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

Преобразуем датасет из `pandas.Series` в наш класс и разделяем выборку на обучающую и валидационную

In [ ]:
dataset = GPT2Dataset(data, tokenizer, max_length=max_len)

train_size = int(len(dataset) * args["train_size"])
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print("train size:", len(train_dataset))
print("validation size:", len(val_dataset))

train size: 8436
validation size: 2110


In [ ]:
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), # Перемешиваем семплы после каждой эпохи
            batch_size = args["batch_size"]
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), # Достаем семплы последовательно
            batch_size = args["batch_size"]
        )

In [3]:
# инициализируем предобученную модель
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# переопределяем токенизатор для модели, так как ранее мы добавили новые токены
model.resize_token_embeddings(len(tokenizer))

# записываем в device устройство, на котором будем обучать модель и загружаем туда саму модель
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# для воспроизводимости определим random seed для всех рандомайзеров в библиотеках
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

В качестве оптимизатора выберем `AdamW` из библиотеки Hugging Face

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = args["learning_rate"],
                  eps = 1e-8,
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Выберем линейный шедулер, который сначала увеличивает learning rate от 0 до заданного в словаре `args`, потом после `num_warmup_steps` шагов линейно уменьшает lr обратно до 0

In [ ]:
total_steps = len(train_dataloader) * args["epochs"]

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = args["warmup_steps"],
                                            num_training_steps = total_steps)

## Цикл обучения модели

В этом цикле во время каждой эпохи модель сначала обучается, затем валидируется. Процесс обучения во во время отдельной эпохи логируется в консоли при помощи библиотеки `tqdm`. После каждой эпохи выводится средние лоссы на трейне и валидации. Если на текущей эпохе лосс на валидации оказался меньше всех предыдущих, веса модели сохраняются на google диск. После всего процесса обучения лоссы на трейне и валидации запишутся в массивы `train_loss` и `val_loss` соответственно.

In [ ]:
train_loss = []
val_loss = []

best_loss = float("inf")

model.to(device)

for epoch_i in range(3):

    total_train_loss = 0

    model.train()
    train_loop = tqdm(train_dataloader, leave=False)

	# обучаем модель 
    for text, mask in train_loop:

        b_input_ids = text.to(device)
        b_labels = text.to(device)
        b_masks = mask.to(device)

        model.zero_grad()

        outputs = model(  b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        loss.backward()

        optimizer.step()
        scheduler.step()

	# лосс на тренеровочной выборке на текущей эпохе 
    avg_train_loss = total_train_loss / len(train_dataloader)

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # валидируем модель
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(b_input_ids,
							# token_type_ids=None,
                            attention_mask = b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

	# лосс на валидации на текущей эпохе 
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), "drive/MyDrive/ml/model_state_dict.pt")

    print(f'Epoch {epoch_i + 1}/{args["epochs"]}, train loss {avg_train_loss:.4f}, val loss {avg_val_loss:.4f}')

    # добавляем лоссы на текущей эпохе в массивы 
    train_loss.append(avg_train_loss)
    val_loss.append(avg_val_loss)

Epoch 1/5, train loss 1.1170, val loss 1.0577


Epoch 2/5, train loss 0.9875, val loss 1.0400


Epoch 3/5, train loss 0.8637, val loss 1.0407


## Генерация текста

Загрузим в модель веса, с которыми она показала лучшие результаты на валидации и напишем функцию для генерации текста.

In [23]:
model.load_state_dict(torch.load("model_state_dict.pt", map_location = "cpu", weights_only=False))

def generate(context = None, 
             num_return_sequences = 1,
             length = 300, 
             no_repeat_ngram_size = 4,
             do_sample = True, 
             temperature = 0.95, 
             top_k = 10, 
             top_p = 0.95) -> list:
  '''
  Генерирует текст песни, возвращает список со строками текстов
  
  Параметры функции:
   - context: начало текста
   - no_repeat_ngram_size: запретить повторение последовательности токенов этого размера
   - num_return_sequences: количество возвращаемых вариантов текстов
   - do_sample: включить вероятностый выбор следующего токена
   - temperature: температура, выравнивает вероятности при большом значении
   - top_k: количество рассматриваемых наиболее вероятных токенов 
   - top_p: параметр nucleus sampling
  '''
  
  # токенизируем тест
  context = tokenizer.encode(context,  return_tensors="pt") if context else None
  
	# генерируем варианты продолжения текста
  output = model.generate(inputs = context,
                        max_length = length, 
                        no_repeat_ngram_size = no_repeat_ngram_size,
                        num_return_sequences = num_return_sequences,
                        temperature = temperature, 
                        top_k=top_k,
    										top_p=top_p, 
                        do_sample=do_sample,
                        pad_token_id=tokenizer.pad_token_id)

	# декодируем каждый текст
  res = [tokenizer.decode(output[i], clean_up_tokenization_spaces=True).replace("<|endoftext|>", "") for i in range(len(output))]
  
  return res


# пробуем сгенерировать продолжение песни Master of Puppets по первому куплету
print(generate(
  					  "End of passion play, crumbling away\n\
               I'm your source of self-destruction\n\
               Veins that pump with fear, sucking darkest clear\n\
               Leading on your death's construction\n"
               )[0])

End of passion play, crumbling away
I'm your source of self-destruction
Veins that pump with fear, sucking darkest clear
Leading on your death's construction
I've got to face the facts, I've got to fight it
I've gotta fight it


I've been waiting for this to happen to me
I've tried to fight my way out

I'm falling in the dark

The sun is always burning
And the sun is always free

I can't believe it's a mistake

I don't believe it
I'm always on the run



A part of me that you've never known
I've never been a victim of the past
I've seen the world and it's not fair

I won't be the victim of the hate that you've become
I've always been waiting for a change to come


In the dark
The sun's always burning

And the Sun is always free


I'm living in a world that's just a shell

I'll fight and never fall

I know what you're about to see
You're never to blame

I have to fight it


I've fallen in the dark



A single word from a thousand times
The sun and the sun


All the sun's always red
I'm 

In [24]:
# теперь попробуем сгенерировать песню с нуля
print(generate()[0])

I can't see the world
We're going down
And the only way out
It's all just a lie
We're losing the race
We're never gonna change
We're the chosen one
We're gonna fight for what is right
You're the chosen ones
We've got a plan to make amends
You know you don't need a reason
We're a chosen one
You'll live for the fight
We're not gonna change
I'm a chosen one 
You'll never change 
We're our chosen one
So don't try to deny 
We've come to save ourselves 
We'll fight to make amenders
You know there's no compromise
We're only here to fight for what's right

You're chosen ones
You'll be born to lose
We're alive
We're fighting for what is wrong
We'll never change
We'll always fight 
You know it's our choice
We'll make amends 
You're not gonna lose
We'll live for this fight
We'll find a way out 
You can't be wrong 
We live for what is real
You'll die before we're born
You're a chosen few
You'll fight for what you believe
You're born to lose 
We stand alone 
We never change
The chosen ones 
We will